Figure des essaies à supprimer

- saccades dans GAP (début GAP - fin GAP + 150ms) > 45ms
- trop de saccade dans poursuite (>= 1/3 de la poursuite)

In [1]:
%load_ext autoreload
%autoreload 2
from __future__ import division, print_function, absolute_import
import pickle
import os
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
from edfreader import read_edf
import numpy as np
from lmfit import  Model, Parameters
%matplotlib inline

In [2]:
def exponentiel (x, tau, maxi, start_anti, v_anti, latence, bino) :
    v_anti = v_anti/1000
    time = np.arange(len(x))
    vitesse = []
    for t in range(len(time)):
        if start_anti >= latence :
            if time[t] < latence :
                vitesse.append(0)
            else :
                vitesse.append((bino*2-1)*maxi*(1-np.exp(-1/tau*(time[t]-latence))))
        else :
            if time[t] < start_anti :
                vitesse.append(0)
            else :
                if time[t] < latence :
                    vitesse.append((time[t]-start_anti)*v_anti)
                    x = (time[t]-start_anti)*v_anti
                else :
                    vitesse.append((bino*2-1)*maxi*(1-np.exp(-1/tau*(time[t]-latence)))+x)
    return vitesse

In [3]:
!ls ../data/*.asc

../data/enregistrement_AM_2017-10-23_100057.asc
../data/enregistrement_BMC_2017-09-26_095637.asc
../data/enregistrement_CS_2017-10-03_134421.asc
../data/enregistrement_DC_2017-09-27_161040.asc
../data/enregistrement_FM_2017-10-03_143803.asc
../data/enregistrement_IP_2017-09-28_115250.asc
../data/enregistrement_LB_2017-09-20_151043 (1).asc
../data/enregistrement_LB_2017-09-20_151043.asc
../data/enregistrement_OP_2017-10-26_121823.asc
../data/enregistrement_RS_2017-11-08_094717.asc
../data/enregistrement_SR_2017-11-16_153313.asc
../data/enregistrement_TN_2017-11-08_150410.asc
../data/enregistrement_YK_2017-11-17_172706.asc


In [4]:
sujet = ['AM','BMC','CS','DC','FM','IP','LB','OP','RS','SR','TN','YK']
time = ['2017-10-23_100057','2017-09-26_095637',
        '2017-10-03_134421','2017-09-27_161040',
        '2017-10-03_143803','2017-09-28_115250',
        '2017-09-20_151043','2017-10-26_121823',
        '2017-11-08_094717','2017-11-16_153313',
        '2017-11-08_150410','2017-11-17_172706']

In [5]:
Liste_suppresion_essais = {}

for x in range(len(sujet)) :
    timeStr = time[x]
    observer = sujet[x]
    mode = 'enregistrement'
    
    print(observer,timeStr)
    
    file = os.path.join('..','data', mode + '_' + observer + '_' + timeStr + '.pkl')
    with open(file, 'rb') as fichier :
        exp = pickle.load(fichier, encoding='latin1')

    resultats = os.path.join('..','data', mode + '_' + observer + '_' + timeStr + '.asc')
    data = read_edf(resultats, 'TRIALID')

    N_trials = exp['N_trials']
    N_blocks = exp['N_blocks']
    p = exp['p']

    Liste_suppresion_essais['%s'%observer] = []
    for block in range(N_blocks) :
        supp_trial = []
        for trial in range(N_trials) :
            bino=p[trial, block, 0]
            trial_data = trial + N_trials*block
            data_x = data[trial_data]['x']
            trackertime = data[trial_data]['trackertime']

            StimulusOf = data[trial_data]['events']['msg'][14][0]
            TargetOn = data[trial_data]['events']['msg'][15][0]
            TargetOff = data[trial_data]['events']['msg'][16][0]
            saccades = data[trial_data]['events']['Esac']

            trackertime_0 = data[trial_data]['trackertime'][0]

            gradient_x = np.gradient(data_x)
            gradient_deg = gradient_x * 1/exp['px_per_deg'] * 1000 # gradient en deg/sec

            ##################################################
            # SUPPRESSION DES SACCADES
            ##################################################
            gradient_deg_NAN = gradient_deg
            s_gap= []
            for s in range(len(saccades)) :
                s_gap.append([])
                if saccades[s][1]-trackertime_0+15 <= (len(trackertime)) : #(TargetOff-trackertime_0-300) : #
                    for x_data in np.arange((saccades[s][0]-trackertime_0-5), (saccades[s][1]-trackertime_0+15)) :
                        gradient_deg_NAN[x_data] = np.nan
                        #if x_data in np.arange(StimulusOf-trackertime_0, TargetOn-trackertime_0+150) :
                        if x_data in np.arange(TargetOn-trackertime_0-150, TargetOn-trackertime_0+150) :
                            s_gap[s].append(x_data)
                '''else :
                    for x_data in np.arange((saccades[s][0]-trackertime_0-5), (len(trackertime))) :
                        gradient_deg_NAN[x_data] = np.nan
                        #if x_data in np.arange(StimulusOf-trackertime_0, TargetOn-trackertime_0+150) :
                        if x_data in np.arange(TargetOn-trackertime_0-150, TargetOn-trackertime_0+150) :
                            s_gap[s].append(x_data)'''
            ##################################################

            # supp grosse saccade GAP
            t = None
            for x in range(len(s_gap)) :
                if len(s_gap[x]) > 45 :
                    if t is None :
                        #print('GAP', block, trial)
                        supp_trial.append(trial)
                        t = True
            if t is None :
                # supp trop saccade poursuite
                lala2 = gradient_deg_NAN[TargetOn-trackertime_0:TargetOff-trackertime_0-300]
                baba2 = [x for x in lala2 if str(x)=='nan']
                if len(baba2) >= len(lala2)/3.6 :
                    #print('poursuite', block, trial)
                    supp_trial.append(trial)
                    '''for s in range(len(saccades)) :
                        if saccades[s][0]-trackertime_0-5 > TargetOn-trackertime_0+350 :
                            print('sacc',((saccades[s][1]-trackertime_0+15))-(saccades[s][0]-trackertime_0-5))'''
                    

        
        print(block, supp_trial)
        Liste_suppresion_essais['%s'%observer].extend([supp_trial])

AM 2017-10-23_100057
0 []
1 []
2 []
BMC 2017-09-26_095637
0 [76, 156]
1 [57, 66, 71, 126, 128, 129, 137]
2 [79, 137]
CS 2017-10-03_134421
0 []
1 []
2 [95]
DC 2017-09-27_161040
0 [101, 120, 185]
1 [49, 66, 85, 127, 130, 144, 164]
2 [31, 36]
FM 2017-10-03_143803
0 [14, 29, 35, 50, 58, 59, 60, 71, 73, 77, 80, 104, 157, 161, 167, 173, 174, 175, 178, 198]
1 [12, 31, 64, 68, 69, 70, 96, 98, 121, 129]
2 [100, 158, 177]
IP 2017-09-28_115250
0 []
1 []
2 []
LB 2017-09-20_151043
0 [10, 42, 82, 83, 143, 150, 151, 163, 179]
1 [72, 116, 123, 162, 164, 178]
2 [5, 11, 21, 32, 44, 53, 98]
OP 2017-10-26_121823
0 [28, 32, 38, 40, 46, 52, 68, 71, 76, 81, 85, 86, 89, 90, 98, 122, 126, 130, 131, 140, 143, 154, 166, 179, 183, 196, 199]
1 [36, 48, 57, 94, 96, 145, 149, 182, 185, 194, 197]
2 [1, 26, 44, 52, 84, 95, 97, 131, 137, 140, 148, 157, 168, 173, 177, 186, 187]
RS 2017-11-08_094717
0 []
1 [30, 180]
2 [85, 90, 170, 177, 190]
SR 2017-11-16_153313
0 [50, 56, 96, 137, 143, 148]
1 [21, 40]
2 [186, 199]
TN 20

In [6]:
file = os.path.join('parametre', 'Liste_suppresion_essaies_150_150.pkl')
with open(file, 'wb') as fichier:
    f = pickle.Pickler(fichier)
    f.dump(Liste_suppresion_essais)

In [7]:
for x in range(len(sujet)) :
    timeStr = time[x]
    observer = sujet[x]
    mode = 'enregistrement'
    
    print(observer,timeStr)
    
    file = os.path.join('..','data', mode + '_' + observer + '_' + timeStr + '.pkl')
    with open(file, 'rb') as fichier :
        exp = pickle.load(fichier, encoding='latin1')

    resultats = os.path.join('..','data', mode + '_' + observer + '_' + timeStr + '.asc')
    data = read_edf(resultats, 'TRIALID')

    N_trials = exp['N_trials']
    N_blocks = exp['N_blocks']
    p = exp['p']

    for block in range(N_blocks) :
        liste_trial = Liste_suppresion_essais['%s'%observer][block]
        print(liste_trial)
        
        if len(liste_trial) != 0 :
            fig_width= 12
            fig, axs = plt.subplots(len(liste_trial), 1, figsize=(fig_width, (fig_width*(len(liste_trial)/2))/1.6180))

            x = 0
            for trial in liste_trial :

                if len(liste_trial)==1 :
                    ax = axs
                else :
                    ax = axs[x]

                print(block, trial)
                trial_data = trial + N_trials*block
                data_x = data[trial_data]['x']
                data_y = data[trial_data]['y']
                trackertime = data[trial_data]['trackertime']

                StimulusOn = data[trial_data]['events']['msg'][10][0]
                StimulusOf = data[trial_data]['events']['msg'][14][0]
                TargetOn = data[trial_data]['events']['msg'][15][0]
                TargetOff = data[trial_data]['events']['msg'][16][0]
                saccades = data[trial_data]['events']['Esac']

                trackertime_0 = data[trial_data]['trackertime'][0]

                gradient_x = np.gradient(data_x)
                gradient_deg = gradient_x * 1/exp['px_per_deg'] * 1000 # gradient en deg/sec

                ##################################################
                # SUPPRESSION DES SACCADES
                ##################################################
                gradient_deg_NAN = gradient_deg
                for s in range(len(saccades)) :
                    if saccades[s][1]-trackertime_0+15 <= (len(trackertime)) :
                        for x_data in np.arange((saccades[s][0]-trackertime_0-5), (saccades[s][1]-trackertime_0+15)) :
                            gradient_deg_NAN[x_data] = np.nan
                    else :
                        for x_data in np.arange((saccades[s][0]-trackertime_0-5), (len(trackertime))) :
                            gradient_deg_NAN[x_data] = np.nan

                stop_latence = []    
                for s in range(len(saccades)) :
                    if (saccades[s][0]-trackertime_0) >= (TargetOn-trackertime_0+100) :
                        stop_latence.append((saccades[s][0]-trackertime_0))
                if stop_latence==[] :
                    stop_latence.append(len(trackertime))
                ##################################################

                start = TargetOn

                StimulusOn_s = StimulusOn - start
                StimulusOf_s = StimulusOf - start
                TargetOn_s = TargetOn - start
                TargetOff_s = TargetOff - start
                trackertime_s = trackertime - start
                
                ##################################################
                # FIT
                ##################################################
                model = Model(exponentiel)
                bino=p[trial, block, 0]
                params = Parameters()
                params.add('tau', value=15., min=13., max=70.)#, vary=False)
                params.add('maxi', value=15., min=1., max=40.)#, vary=False)
                params.add('latence', value=TargetOn-trackertime_0+100, min=TargetOn-trackertime_0+75, max=stop_latence[0])
                params.add('start_anti', value=TargetOn-trackertime_0-100, min=StimulusOf-trackertime_0-200, max=TargetOn-trackertime_0+75)
                params.add('v_anti', value=0., min=-40., max=40.)
                params.add('bino', value=bino, min=0, max=1, vary=False)

                result_deg = model.fit(gradient_deg_NAN[:-280], params, x=trackertime[:-280], fit_kws={'nan_policy': 'omit'})
                ##################################################

                ax.cla() # pour remettre ax figure a zero
                ax.axis([StimulusOn_s-10, TargetOff_s+10, -40, 40])
                ax.xaxis.set_ticks(range(StimulusOf_s-199, TargetOff_s+10, 500))
                

                ax.plot(trackertime_s, gradient_deg_NAN, color='k', alpha=0.6)
                ax.plot(trackertime_s[:-280], result_deg.init_fit, 'r--', linewidth=2)
                ax.plot(trackertime_s[:-280], result_deg.best_fit, color='r', linewidth=2)
                ax.plot(trackertime_s, np.ones(np.shape(trackertime_s)[0])*(bino*2-1)*(15), color='k', linewidth=0.2, alpha=0.2)
                ax.plot(trackertime_s, np.ones(np.shape(trackertime_s)[0])*(bino*2-1)*(10), color='k', linewidth=0.2, alpha=0.2)
                ax.axvspan(StimulusOn_s, StimulusOf_s, color='k', alpha=0.2)
                ax.axvspan(StimulusOf_s, TargetOn_s, color='r', alpha=0.2)
                ax.axvspan(TargetOn_s, TargetOff_s, color='k', alpha=0.15)
                for s in range(len(saccades)) :
                    ax.axvspan(saccades[s][0]-start, saccades[s][1]-start, color='k', alpha=0.2)

                debut  = TargetOn - trackertime_0 # TargetOn - temps_0

                start_anti = result_deg.values['start_anti']-debut
                v_anti = result_deg.values['v_anti']
                latence = result_deg.values['latence']-debut
                tau = result_deg.values['tau']
                maxi = result_deg.values['maxi']

                ax.bar(latence, 80, bottom=-40, color='r', width=6, linewidth=0)

                if x==0 :
                    ax.text(StimulusOn_s+(StimulusOf_s-StimulusOn_s)/2, 31, "FIXATION", color='k', fontsize=16, ha='center', va='bottom')
                    ax.text(StimulusOf_s+(TargetOn_s-StimulusOf_s)/2, 31, "GAP", color='r', fontsize=16, ha='center', va='bottom')
                    ax.text(TargetOn_s+(TargetOff_s-TargetOn_s)/2, 31, "POURSUITE", color='k', fontsize=16, ha='center', va='bottom')
                    ax.text(latence+25, -35, "Latence"%(latence), color='r', fontsize=14)#,  weight='bold')
                ax.text(StimulusOn_s+15, 18, "start_anti: %s \nv_anti: %s"%(start_anti, v_anti), color='k', fontsize=14, va='bottom')
                ax.text(StimulusOn_s+15, -18, "latence: %s \ntau: %s \nmaxi: %s"%(latence, tau, maxi), color='k', fontsize=14, va='top')

                ax.set_xlabel('Time (ms)', fontsize=9)
                ax.set_ylabel(trial+1, fontsize=30)
                ax.xaxis.set_ticks_position('bottom')
                ax.yaxis.set_ticks_position('left')
                
                x=x+1

            plt.tight_layout() # pour supprimer les marge trop grande
            plt.subplots_adjust(hspace=0) # pour enlever espace entre les figures

            plt.savefig('figures/suppresion/suppresion_%s_%s.pdf'%(observer, block+1))
            plt.close()

    print('FIN !!!')

AM 2017-10-23_100057
[]
[]
[]
FIN !!!
BMC 2017-09-26_095637
[76, 156]
0 76
0 156
[57, 66, 71, 126, 128, 129, 137]
1 57
1 66
1 71
1 126
1 128
1 129
1 137
[79, 137]
2 79
2 137
FIN !!!
CS 2017-10-03_134421
[]
[]
[95]
2 95
FIN !!!
DC 2017-09-27_161040
[101, 120, 185]
0 101
0 120
0 185
[49, 66, 85, 127, 130, 144, 164]
1 49
1 66
1 85
1 127
1 130
1 144
1 164
[31, 36]
2 31
2 36
FIN !!!
FM 2017-10-03_143803
[14, 29, 35, 50, 58, 59, 60, 71, 73, 77, 80, 104, 157, 161, 167, 173, 174, 175, 178, 198]
0 14
0 29
0 35
0 50
0 58
0 59
0 60
0 71
0 73
0 77
0 80
0 104
0 157
0 161
0 167
0 173
0 174
0 175
0 178
0 198
[12, 31, 64, 68, 69, 70, 96, 98, 121, 129]
1 12
1 31
1 64
1 68
1 69
1 70
1 96
1 98
1 121
1 129
[100, 158, 177]
2 100
2 158
2 177
FIN !!!
IP 2017-09-28_115250
[]
[]
[]
FIN !!!
LB 2017-09-20_151043
[10, 42, 82, 83, 143, 150, 151, 163, 179]
0 10
0 42
0 82
0 83
0 143
0 150
0 151
0 163
0 179
[72, 116, 123, 162, 164, 178]
1 72
1 116
1 123
1 162
1 164
1 178
[5, 11, 21, 32, 44, 53, 98]
2 5
2 11
2 21
2 32